## Env set-up

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# For emptying trash after each run
#from google.colab import auth
#auth.authenticate_user()
#from googleapiclient.discovery import build
#drive_service = build('drive', 'v3')
#drive_service.files().emptyTrash().execute()

In [3]:
!pwd
!nvidia-smi

/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/subtask1/transformer/inference
Sun Jan 28 20:56:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  | 00000000:2B:00.0 Off |                  N/A |
|  0%   35C    P8              26W / 350W |   5060MiB / 12288MiB |      0%      Default |
|                                         |                    

In [4]:
#folder_name = "/content/drive/MyDrive/persuasion_technique_detection/" 
folder_name = "/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/"
summary_dir_path = folder_name + "subtask1/transformer/summaries/inference/hierarchy_inference_originalScorer/"

In [5]:
#!pip install transformers datasets wandb evaluate accelerate -qU sklearn_hierarchical_classification sentencepiece

In [6]:
import gc
import re
import json
import numpy as np
import pandas as pd
import random
import torch
import subprocess
import json
import warnings
import shutil
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
from datasets import Dataset,load_dataset,DatasetDict,concatenate_datasets
import datasets
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
import os
from datasets import concatenate_datasets
from transformers import Trainer
from transformers import AutoTokenizer, DataCollatorWithPadding

In [7]:
AVAIL_GPUS = 0
if torch.cuda.is_available():
    device = torch.device("cuda")
    AVAIL_GPUS = torch.cuda.device_count()
    print(f'There are {AVAIL_GPUS} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3080 Ti


In [8]:
import wandb
import os

#wandb.login(relogin=True)
wandb.login()

# setup wandb environment variables
os.environ['WANDB_PROJECT'] = "subtask1_transformer_encoder_classification"
os.environ['WANDB_ENTITY'] = "tumnlp"
os.environ["WANDB_LOG_MODEL"]= "end"

wandb: Currently logged in as: mahmudfami (tumnlp). Use `wandb login --relogin` to force relogin


## Data Importing and Preprocessing

In [9]:
val_path=folder_name+"data/subtask1/validation.json"
train_path = folder_name+"data/subtask1/train.json"
test_path=folder_name+"data/subtask1/dev_unlabeled.json"

train_files={"train":train_path}
val_files={"val":val_path}
test_files={"test":test_path}

dataset_train=load_dataset("json", data_files=train_files)
dataset_val=load_dataset("json", data_files=val_files)
dataset_test=load_dataset("json",data_files=test_files)

In [10]:
persuasion_unique=[["Ethos","Pathos","Logos"]]
ethos_unique=[["Ad Hominem","Bandwagon","Appeal to authority","Glittering generalities (Virtue)"]]
pathos_unique=[["Exaggeration/Minimisation","Loaded Language","Flag-waving","Appeal to fear/prejudice"]]
logos_unique=[["Justification","Reasoning","Repetition","Obfuscation, Intentional vagueness, Confusion"]]
ad_hominem_unique=[["Name calling/Labeling","Doubt","Smears","Reductio ad hitlerum","Whataboutism"]]
justification_unique=[["Bandwagon","Appeal to authority","Flag-waving","Appeal to fear/prejudice","Slogans"]]
reasoning_unique=[["Distraction","Simplification"]]
distraction_unique=[["Misrepresentation of Someone's Position (Straw Man)","Presenting Irrelevant Data (Red Herring)","Whataboutism"]]
simplification_unique=[["Causal Oversimplification","Black-and-white Fallacy/Dictatorship","Thought-terminating cliché"]]
persuasion_or_not_unique=[["Not Persuasion","Persuasion"]]

mlb_persuasion = MultiLabelBinarizer()
mlb_persuasion.fit(persuasion_unique)

mlb_ethos = MultiLabelBinarizer()
mlb_ethos.fit(ethos_unique)

mlb_pathos = MultiLabelBinarizer()
mlb_pathos.fit(pathos_unique)

mlb_logos = MultiLabelBinarizer()
mlb_logos.fit(logos_unique)

mlb_ad_hominem = MultiLabelBinarizer()
mlb_ad_hominem.fit(ad_hominem_unique)

mlb_justification = MultiLabelBinarizer()
mlb_justification.fit(justification_unique)

mlb_reasoning=MultiLabelBinarizer()
mlb_reasoning.fit(reasoning_unique)

mlb_distraction=MultiLabelBinarizer()
mlb_distraction.fit(distraction_unique)

mlb_simplification=MultiLabelBinarizer()
mlb_simplification.fit(simplification_unique)

lb_persuasion_or_not=LabelEncoder()
lb_persuasion_or_not.fit(persuasion_or_not_unique[0])


LabelEncoder()

In [11]:
def write_json(path,data,test=False):
  if not isinstance(data, dict):
    data = data.to_dict("records")

  with open(path, "w") as output_file:
      json.dump(data, output_file, indent=2,ensure_ascii=False)

## Save multi models in a summary json

In [12]:
api=wandb.Api()
runs=api.runs("subtask1_transformer_encoder_classification")

In [13]:
def get_summaries(path):
  l=[]
  for root, _, files in os.walk(path):
    for file in files:
      if "summary" in file:
        file_name=os.path.join(root, file)
        with open(file_name, 'r') as f:
          l.append(json.load(f))
  return l

node_names = ["persuasion_or_not","persuasion","ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
summary_files=get_summaries(folder_name+"subtask1/transformer/summaries/hierarchical")
model_nodes= {
    "persuasion_or_not": "model-vinai_bertweet-large-persuasion_or_not-memes_0.5threshold_5e-06learningRate:v12",
    "persuasion": "model-vinai_bertweet-large-persuasion-memes_0.3threshold_5e-06learningRate:v6",
    "ethos": "model-vinai_bertweet-large-ethos-memes_0.4threshold_5e-06learningRate:v9",
    "pathos": "model-vinai_bertweet-large-pathos-memes_0.3threshold_5e-06learningRate:v5",
    "logos": "model-vinai_bertweet-large-logos-memes_0.5threshold_5e-06learningRate:v5",
    "ad_hominem": "model-vinai_bertweet-large-ad_hominem-memes_0.3threshold_5e-06learningRate:v5",
    "justification": "model-vinai_bertweet-large-justification-memes_0.4threshold_5e-06learningRate:v8",
    "reasoning": "model-vinai_bertweet-large-reasoning-memes_0.3threshold_5e-06learningRate:v5",
    "simplification": "model-vinai_bertweet-large-simplification-memes_0.4threshold_5e-06learningRate:v6",
    "distraction": "model-vinai_bertweet-large-distraction-memes_0.5threshold_5e-06learningRate:v0"
  }

threshold_nodes= {
    "persuasion_or_not": 0.5,
    "persuasion": 0.3,
    "ethos": 0.4,
    "pathos": 0.3,
    "logos": 0.5,
    "ad_hominem": 0.3,
    "justification": 0.4,
    "reasoning": 0.3,
    "simplification": 0.4,
    "distraction": 0.5
  }
checkpoints= {
    "persuasion_or_not": "vinai/bertweet-large",
    "persuasion": "vinai/bertweet-large",
    "ethos": "vinai/bertweet-large",
    "pathos": "vinai/bertweet-large",
    "logos": "vinai/bertweet-large",
    "ad_hominem": "vinai/bertweet-large",
    "justification": "vinai/bertweet-large",
    "reasoning": "vinai/bertweet-large",
    "simplification": "vinai/bertweet-large",
    "distraction": "vinai/bertweet-large"
}

best_multiple_models={"model_nodes": model_nodes,"threshold_nodes":threshold_nodes,"checkpoints":checkpoints}
best_multiple_models

{'model_nodes': {'persuasion_or_not': 'model-vinai_bertweet-large-persuasion_or_not-memes_0.5threshold_5e-06learningRate:v12',
  'persuasion': 'model-vinai_bertweet-large-persuasion-memes_0.3threshold_5e-06learningRate:v6',
  'ethos': 'model-vinai_bertweet-large-ethos-memes_0.4threshold_5e-06learningRate:v9',
  'pathos': 'model-vinai_bertweet-large-pathos-memes_0.3threshold_5e-06learningRate:v5',
  'logos': 'model-vinai_bertweet-large-logos-memes_0.5threshold_5e-06learningRate:v5',
  'ad_hominem': 'model-vinai_bertweet-large-ad_hominem-memes_0.3threshold_5e-06learningRate:v5',
  'justification': 'model-vinai_bertweet-large-justification-memes_0.4threshold_5e-06learningRate:v8',
  'reasoning': 'model-vinai_bertweet-large-reasoning-memes_0.3threshold_5e-06learningRate:v5',
  'simplification': 'model-vinai_bertweet-large-simplification-memes_0.4threshold_5e-06learningRate:v6',
  'distraction': 'model-vinai_bertweet-large-distraction-memes_0.5threshold_5e-06learningRate:v0'},
 'threshold_n

## Multi model evaluation

In [14]:
def tokenize(examples,max_length=None):

  if max_length is not None:

    encoding = tokenizer(examples["text"], truncation=True,padding=True,max_length=max_length)
  else:
    encoding = tokenizer(examples["text"], truncation=True,padding=True)
  return encoding

def get_preds(binarizer,predicted_logits,threshold=0.5):

  predicted_logits_tensor = torch.from_numpy(predicted_logits)

  if isinstance(binarizer,LabelEncoder):
    softmax=torch.nn.Softmax(dim=1)
    full_probs=softmax(predicted_logits_tensor)

    probs,predictions=torch.max(full_probs,dim=1)
    predictions=[[p] for p in binarizer.inverse_transform(predictions)]
    probs = [[value.item()] for value in probs]
    res=[(x, y) for x, y in zip(predictions, probs)]
    return res
  else:
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(predicted_logits_tensor)
    preds=(probs>threshold).int()
    

    mask = (preds == 1)

    # Use the mask to select the corresponding probabilities
    selected_probs_list = []

    # Iterate over rows and select probabilities using the mask
    for i in range(preds.size(0)):
        selected_probs_row = probs[i][mask[i]].tolist()
        selected_probs_list.append(selected_probs_row)

    names=binarizer.inverse_transform(preds)
    res=[(list(x), y) for x, y in zip(names, selected_probs_list)]
    return res

project_name="subtask1_transformer_encoder_classification"

def return_trainer(model_name,unique_labels):
  api = wandb.Api()
  artifact=api.artifact(model_name)
  model_dir=artifact.download()
  if "persuasion_or_not" in model_name:
    lbls=unique_labels[0]
    id2label={0:lbls[0],1:lbls[1]}
    label2id={lbls[0]:0,lbls[1]:1}
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=len(unique_labels[0]),id2label=id2label, label2id=label2id)
  else:
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=len(unique_labels[0]), problem_type="multi_label_classification", ignore_mismatched_sizes=True)

  trainer = Trainer(model=model)
  if torch.cuda.is_available():
    trainer.model = model.cuda()
  return trainer



### Val set eval

In [15]:
prediction_set=dataset_val["val"]
prediction_set=prediction_set.remove_columns(["labels"])

In [16]:
def get_new_labels(model_address,node,unique,threshold,binarizer):
  trainer=return_trainer(model_address,unique)
  preds=get_preds(binarizer,trainer.predict(curr_set).predictions,threshold)

  for i in range(len(preds)):

    if len(preds[i][0])>0:

      t=prev_labels[i][0].index(node)
      _=prev_labels[i][1].pop(t)
      prev_labels[i][0].remove(node)
      parent_labels=prev_labels[i][0]
      parent_probs=prev_labels[i][1]

      new_labels.append((parent_labels+preds[i][0],parent_probs+preds[i][1]))
    else:
      new_labels.append(prev_labels[i])

In [17]:
trainer=return_trainer(best_multiple_models["model_nodes"]["persuasion_or_not"],persuasion_or_not_unique)
threshold=best_multiple_models["threshold_nodes"]['persuasion_or_not']
unique_models=list(set(best_multiple_models["checkpoints"].values()))
dict_prediction_set_tokenized=dict()

for m in unique_models:
  tokenizer = AutoTokenizer.from_pretrained(m)
  if m=="vinai/bertweet-large" or m=="microsoft/deberta-v3-base":
    
    prediction_set_tokenized=prediction_set.map(tokenize ,fn_kwargs={"max_length":512},batched=True)
  else:
    
    prediction_set_tokenized=prediction_set.map(tokenize ,batched=True)

  dict_prediction_set_tokenized[m]=prediction_set_tokenized

dataset=dict_prediction_set_tokenized[best_multiple_models["checkpoints"]["persuasion_or_not"]]
preds_=get_preds(lb_persuasion_or_not,trainer.predict(dataset).predictions,threshold)
preds=[el if el[0][0]=="Persuasion" else [] for el in preds_]
final_ds=dict(zip(dataset["id"],preds))

model_seq=["Persuasion","Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["persuasion","ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [persuasion_unique, ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_persuasion, mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

wandb: Downloading large artifact model-vinai_bertweet-large-persuasion_or_not-memes_0.5threshold_5e-06learningRate:v12, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:2.8


In [18]:
for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds[x] for x in ids]
  new_labels=[]
  dataset=dict_prediction_set_tokenized[best_multiple_models["checkpoints"][node_names[i]]]
  curr_set=dataset.filter(lambda x: x["id"] in ids)

  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  tokenizer = AutoTokenizer.from_pretrained(best_multiple_models["checkpoints"][node_names[i]])
  get_new_labels(best_multiple_models["model_nodes"][node_names[i]],node,unique_node_labels[i],best_multiple_models["threshold_nodes"][node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds[ids[k]]=new_labels[k]

top_k=9
final_df=dict()
for k,v in final_ds.items():
  if len(v)>0:
    pred_prob=list(zip(v[0],v[1]))
    pred_prob.sort(key=lambda x: -x[1])

    tmp=[]
    i=0
    for el in pred_prob:
      if i==top_k:
        break
      if el[0] not in tmp:
        tmp.append(el[0])
        i+=1
    final_df[str(k)]=tmp
  else:
    final_df[str(k)]=v

pred_df=pd.DataFrame.from_dict({"id":final_df.keys(),"labels":final_df.values()})

val_pred_file=summary_dir_path + "val_pred.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)

write_json(val_pred_file,pred_df)

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Node: Persuasion, curr_set: Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 400
})


wandb: Downloading large artifact model-vinai_bertweet-large-persuasion-memes_0.3threshold_5e-06learningRate:v6, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:2.9


Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Node: Ethos, curr_set: Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 349
})


...

Node: Simplification, curr_set: Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 149
})


wandb: Downloading large artifact model-vinai_bertweet-large-simplification-memes_0.4threshold_5e-06learningRate:v6, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:2.8


Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Node: Distraction, curr_set: Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 24
})


wandb: Downloading large artifact model-vinai_bertweet-large-distraction-memes_0.5threshold_5e-06learningRate:v0, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:2.8


In [19]:
final_df

{'63135': ['Repetition', 'Smears', 'Black-and-white Fallacy/Dictatorship'],
 '78590': [],
 '65834': [],
 '67394': [],
 '67709': ['Appeal to authority',
  'Smears',
  'Black-and-white Fallacy/Dictatorship'],
 '66205': ['Appeal to authority', 'Flag-waving', 'Loaded Language'],
 '79029': ['Flag-waving',
  'Black-and-white Fallacy/Dictatorship',
  'Name calling/Labeling',
  'Loaded Language',
  'Smears'],
 '78559': ['Smears', 'Flag-waving', 'Name calling/Labeling'],
 '77741': [],
 '63599': ['Black-and-white Fallacy/Dictatorship', 'Smears'],
 '77552': ['Appeal to authority',
  'Smears',
  'Name calling/Labeling',
  'Loaded Language'],
 '71273': ['Loaded Language', 'Appeal to fear/prejudice', 'Bandwagon'],
 '65211': ['Smears'],
 '71723': ['Name calling/Labeling',
  'Loaded Language',
  'Black-and-white Fallacy/Dictatorship',
  'Thought-terminating cliché',
  'Smears'],
 '70725': ['Appeal to authority', 'Exaggeration/Minimisation', 'Flag-waving'],
 '65147': ['Smears',
  'Loaded Language',
  '

In [20]:
scorer = folder_name + "subtask1/subtask_1_2a_original.py"
command = f'python3 {scorer} --gold_file_path {val_path} --pred_file_path {val_pred_file}'

result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True)
output = result.stdout.strip()
print(result)

parts = output.split('\t')
f1_h = parts[0].split('=')[1]
prec_h = parts[1].split('=')[1]
rec_h = parts[2].split('=')[1]

f1_h = float(f1_h)
prec_h = float(prec_h)
rec_h = float(rec_h)

hierarchical_metrics = {"f1_hierarchical": f1_h, "precision_hierarchical": prec_h, "recall_hierarchical": rec_h}
hierarchical_metrics

CompletedProcess(args='python3 /home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/subtask1/subtask_1_2a_original.py --gold_file_path /home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/data/subtask1/validation.json --pred_file_path /home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/subtask1/transformer/summaries/inference/hierarchy_inference_originalScorer/val_pred.json', returncode=0, stdout='f1_h=0.60738\tprec_h=0.53450\trec_h=0.70328\n')


{'f1_hierarchical': 0.60738,
 'precision_hierarchical': 0.5345,
 'recall_hierarchical': 0.70328}

In [21]:
best_multiple_models["top_k"] = top_k
best_multiple_models["Validation score"] = hierarchical_metrics
best_multiple_models_file_path=summary_dir_path+"summary.json"
write_json(best_multiple_models_file_path, best_multiple_models)

## Dev set eval

In [22]:
prediction_set=dataset_test["test"]

In [23]:
trainer=return_trainer(best_multiple_models["model_nodes"]["persuasion_or_not"],persuasion_or_not_unique)
threshold=best_multiple_models["threshold_nodes"]['persuasion_or_not']
unique_models=list(set(best_multiple_models["checkpoints"].values()))
dict_prediction_set_tokenized=dict()

for m in unique_models:
  tokenizer = AutoTokenizer.from_pretrained(m)
  if m=="vinai/bertweet-large" or m=="microsoft/deberta-v3-base":

    prediction_set_tokenized=prediction_set.map(tokenize ,fn_kwargs={"max_length":512},batched=True)
  else:
 
    prediction_set_tokenized=prediction_set.map(tokenize ,batched=True)

  dict_prediction_set_tokenized[m]=prediction_set_tokenized

dataset=dict_prediction_set_tokenized[best_multiple_models["checkpoints"]["persuasion_or_not"]]
preds_=get_preds(lb_persuasion_or_not,trainer.predict(dataset).predictions,threshold)
preds=[el if el[0][0]=="Persuasion" else [] for el in preds_]
final_ds=dict(zip(dataset["id"],preds))

model_seq=["Persuasion","Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["persuasion","ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [persuasion_unique, ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_persuasion, mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds[x] for x in ids]
  new_labels=[]
  dataset=dict_prediction_set_tokenized[best_multiple_models["checkpoints"][node_names[i]]]
  curr_set=dataset.filter(lambda x: x["id"] in ids)

  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  tokenizer = AutoTokenizer.from_pretrained(best_multiple_models["checkpoints"][node_names[i]])
  get_new_labels(best_multiple_models["model_nodes"][node_names[i]],node,unique_node_labels[i],best_multiple_models["threshold_nodes"][node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds[ids[k]]=new_labels[k]

top_k=9
final_df=dict()
for k,v in final_ds.items():
  if len(v)>0:
    pred_prob=list(zip(v[0],v[1]))
    pred_prob.sort(key=lambda x: -x[1])

    tmp=[]
    i=0
    for el in pred_prob:
      if i==top_k:
        break
      if el[0] not in tmp:
        tmp.append(el[0])
        i+=1
    final_df[str(k)]=tmp
  else:
    final_df[str(k)]=v

pred_df=pd.DataFrame.from_dict({"id":final_df.keys(),"labels":final_df.values()})

dev_pred_file=summary_dir_path + "dev_pred.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)

write_json(dev_pred_file,pred_df)

wandb: Downloading large artifact model-vinai_bertweet-large-persuasion_or_not-memes_0.5threshold_5e-06learningRate:v12, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:1.8


Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Node: Persuasion, curr_set: Dataset({
    features: ['text', 'id', 'input_ids', 'attention_mask'],
    num_rows: 795
})


wandb: Downloading large artifact model-vinai_bertweet-large-persuasion-memes_0.3threshold_5e-06learningRate:v6, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:1.7


Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

Node: Simplification, curr_set: Dataset({
    features: ['text', 'id', 'input_ids', 'attention_mask'],
    num_rows: 317
})


wandb: Downloading large artifact model-vinai_bertweet-large-simplification-memes_0.4threshold_5e-06learningRate:v6, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:1.8


Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Node: Distraction, curr_set: Dataset({
    features: ['text', 'id', 'input_ids', 'attention_mask'],
    num_rows: 67
})


wandb: Downloading large artifact model-vinai_bertweet-large-distraction-memes_0.5threshold_5e-06learningRate:v0, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:1.8
